In [262]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from dotenv import load_dotenv
import os

In [263]:
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

In [264]:
file_path = rf"{download_path}\Base Consolidada 2025.xlsx"

In [265]:
email_remetente = "projetos@polijunior.com.br"
load_dotenv(dotenv_path='.venv/.env')
senha = 'projetos.pj'

In [266]:
error_log = []

In [267]:
def send_email(remetente, destinatario, assunto, corpo, senha):
        try:
                # Criação da mensagem
                msg = MIMEMultipart()
                msg['From'] = remetente
                msg['To'] = destinatario
                msg['Subject'] = assunto

                # Corpo do e-mail
                msg.attach(MIMEText(corpo, 'html', 'utf-8'))

                # Conectar ao servidor SMTP do Gmail
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()  # Encriptação TLS

                # Login no servidor
                server.login(remetente, senha)

                # Envio do e-mail
                text = msg.as_string()
                server.sendmail(remetente, destinatario, text)

                # Encerrar a conexão
                server.quit()

                print(f"E-mail enviado com sucesso para {destinatario}")
        except Exception as e:
                error_log.append(f"Falha ao enviar e-mail para {destinatario}: {e}")

# Feeds quinzenais

In [268]:
feedbacks_quinzenais = pd.read_excel(file_path, sheet_name='Forms Quinzenal de Projetos')

In [269]:
def obter_ultima_sexta():
    hoje = pd.Timestamp.today()
    # Obter a última sexta-feira usando a funcionalidade de dias úteis do pandas
    ultima_sexta = hoje - pd.tseries.offsets.Week(weekday=4)
    return ultima_sexta.replace(hour=0, minute=0, second=0, microsecond=0)

In [270]:
hoje = pd.Timestamp.today()
# Obter a última sexta-feira usando a funcionalidade de dias úteis do pandas
ultima_sexta = hoje - pd.tseries.offsets.Week(weekday=4)
ultima_sexta = ultima_sexta.replace(hour=0, minute=0, second=0, microsecond=0)

In [271]:
# Filtre o DataFrame para mostrar apenas datas a partir da última sexta-feira
feedbacks_quinzenais = feedbacks_quinzenais.dropna(subset='Timestamp').reset_index()
feedbacks_quinzenais = feedbacks_quinzenais[feedbacks_quinzenais['A qual semana as respostas a seguir se referem?\nSelecione a sexta-feira da quinzena à qual se refere.'] >= ultima_sexta]

In [272]:
def cria_corpo_email_analista(projeto, nucleo, nota_qualidade, nota_compromisso, nota_comunicacao, feedback):

    assunto = f'[FEEDBACKS] - {projeto}'

    corpo = f'''<!DOCTYPE html>
        <html lang="pt">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
        </head>
        <corpo>
            <h1>Feedbacks: {projeto}</h1>

            <p><strong>Núcleo:</strong> {nucleo}</p>
            <p><strong>Nota de Qualidade da Entrega:</strong> {nota_qualidade}</p>
            <p><strong>Nota de Compromisso:</strong> {nota_compromisso}</p>
            <p><strong>Nota de Comunicação:</strong> {nota_comunicacao}</p>
            <p><strong>Feedback geral:</strong> {feedback}</p>
            Não deixe de conversar com sua liderança para discutir e entender mais sobre os feedbacks!
        </corpo>
        </html>'''
    
    return assunto, corpo

def obter_dados_analista(linha, numero_analista):
    """Extrair dados de um analista com base no seu número"""
    if numero_analista == 1:
        # O primeiro analista tem nomes de colunas diferentes
        return {
            'email': linha['Analista 1'] + "@polijunior.com.br" if pd.notna(linha['Analista 1']) else None,
            'qualidade': linha['Analista 1 [Qualidade de entrega]'],
            'compromisso': linha['Analista 1 [Compromisso com o projeto]'],
            'comunicacao': linha['Analista 1 [Comunicação e dinâmica de trabalho]'],
            'feedback': linha['Dê o feedback ao analista sobre a nota recebida']
        }
    else:
        return {
            'email': linha[f'Analista {numero_analista}.1'] + "@polijunior.com.br" if pd.notna(linha[f'Analista {numero_analista}.1']) else None,
            'qualidade': linha[f'Analista {numero_analista} [Qualidade das entregas]'],
            'compromisso': linha[f'Analista {numero_analista} [Compromisso com o projeto]'],
            'comunicacao': linha[f'Analista {numero_analista} [Comunicação e dinâmica de trabalho]'],
            'feedback': linha[f'Dê o feedback ao analista sobre a nota recebida.{numero_analista-1}']
        }
        
def obter_nome_projeto(linha):
    """Extrair o nome do projeto da primeira coluna de projeto não nula"""
    colunas_projeto = [
        'Nome do Projeto NCiv', 
        'Nome do Projeto NTec', 
        'Nome do Projeto NDados', 
        'Nome do Projeto NCon'
    ]
    
    for coluna in colunas_projeto:
        if pd.notna(linha[coluna]):
            return linha[coluna]
    return None

In [273]:
lista_emails = []

In [274]:
for indice, linha in feedbacks_quinzenais.iterrows():
    projeto = obter_nome_projeto(linha)
    nucleo = linha['Núcleo']
    
    for i in range(0, 4): #ate 4 analistas
        try:
            dados_analista = obter_dados_analista(linha, i)
            
            if not dados_analista['email']:
                continue
                
            titulo, corpo = cria_corpo_email_analista(
                projeto=projeto,
                nucleo=nucleo,
                nota_qualidade=dados_analista['qualidade'],
                nota_compromisso=dados_analista['compromisso'],
                nota_comunicacao=dados_analista['comunicacao'],
                feedback=dados_analista['feedback']
            )
            email_analista = dados_analista['email']
            lista_emails.append({
                'destinario': email_analista,
                'assunto': titulo,
                'corpo': corpo
            })
            
            print(f"E-mail preparado para {email_analista}")
                  
        except Exception as e:
            error_log.append(f"Erro ao processar o analista {i} na linha {indice}: {e}")
            break

# TESTE

In [275]:
email_teste = "gabriel.agra@polijunior.com.br"

In [276]:
# Se quiser mandar todos, use a linha abaixo
# for email in lista_emails:

# Mandamos 2 e-mails de teste
for i in range(min(2, len(lista_emails))):
    email = lista_emails[i]
    send_email(
        remetente=email_remetente,
        destinatario=email_teste,  # Manda para email de teste
        assunto=f"[TESTE - {email['destinario']}]: {email['assunto']}", 
        corpo=email['corpo'],
        senha=senha
    )

if error_log:
    print("\nErros encontrados:")
    for erro in error_log:
        print(f"- {erro}")

# E-MAIL EM MASSA

In [277]:
for email in lista_emails:
    send_email(
        remetente=email_remetente,
        destinatario=email['email_analista'],  # Manda para email do analista
        assunto=email['assunto'], 
        corpo=email['corpo'],
        senha=senha
    )

if error_log:
    print("\nErros encontrados:")
    for erro in error_log:
        print(f"- {erro}")